## 以下所有程式皆為參考 https://www.itread01.com/content/1549244008.html
我們只調整參數盡量優化結果

### import library、建立讀入資料的前處理generator(包括normalize、一些augmentation)：

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import GlobalAveragePooling2D,Dense
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.optimizers import Adagrad
# 資料準備
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,# ((x/255)-0.5)*2  歸一化到±1之間
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

### 調整參數：

In [ ]:
train_directory = './train'
test_directory = './test'


set_GAP_LAYER =  #第幾層之上全凍結，其之下全部解凍
epochs = 
learning_rate = 0.0001 # 解凍後的learning rate
num_of_output =  #output幾種類別
training_batch_size =  #一個training_batch的資料個數
validation_batch_size =  #一個validation_batch的資料個數

training_total_file_number =  #總共training資料個數
validate_total_file_number =  #總共testing資料個數

# https://www.codespeedy.com/how-to-set-steps-per-epoch-with-keras/
import math
def calculate_spe(y, batch_size):
    return int(math.ceil((1. * y) / batch_size)) 
steps_per_epoch = calculate_spe(training_total_file_number, training_batch_size)
validation_steps = calculate_spe(validate_total_file_number, validation_batch_size)

### 建立之後model.fit_generator要輸入的mapping資料(減少RAM占用)：

In [ ]:
train_generator = train_datagen.flow_from_directory(directory=train_directory,
                                  target_size=(299,299),#Inception V3規定大小 299*299
                                  batch_size=training_batch_size)
val_generator = val_datagen.flow_from_directory(directory=test_directory,
                                target_size=(299,299),
                                batch_size=validation_batch_size)

### 沿用inception v3模型和訓練好的權重，並在其尾端接上自訂output layer：

In [ ]:
# 構建基礎模型
base_model = InceptionV3(weights='imagenet',include_top=False)

# 增加新的輸出層
x = base_model.output
x = GlobalAveragePooling2D()(x) # GlobalAveragePooling2D 將 MxNxC 的張量轉換成 1xC 張量，C是通道數
x = Dense(1024,activation='relu')(x)
predictions = Dense(17,activation='softmax')(x)
model = Model(inputs=base_model.input,outputs=predictions)

### 調整解凍範圍：

In [ ]:
def setup_to_transfer_learning(model,base_model):#base_model
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

def setup_to_fine_tune(model,base_model):
    GAP_LAYER = set_GAP_LAYER
    for layer in base_model.layers[:GAP_LAYER+1]:
        layer.trainable = False
    for layer in base_model.layers[GAP_LAYER+1:]:
        layer.trainable = True
    model.compile(optimizer=Adagrad(lr=learning_rate),loss='categorical_crossentropy',metrics=['accuracy'])

### 訓練：

In [ ]:
setup_to_transfer_learning(model,base_model)
history_tl = model.fit_generator(generator=train_generator,
                    steps_per_epoch=steps_per_epoch,
                    epochs=epochs,
                    validation_data=val_generator,
                    validation_steps=validation_steps,
                    class_weight='auto'
                    )
model.save('./flowers17_iv3_tl.h5')
setup_to_fine_tune(model,base_model)
history_ft = model.fit_generator(generator=train_generator,
                                 steps_per_epoch=steps_per_epoch,
                                 epochs=epochs,
                                 validation_data=val_generator,
                                 validation_steps=validation_steps,
                                 class_weight='auto')
model.save('./flowers17_iv3_ft.h5')

### 繪製accuracy和loss對epoch折線圖：

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 繪製confusion_matrix：

In [ ]:
#這些還要改，不能直接用!!
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

pred = model.predict(X_test)
pred = np.argmax(pred,axis = 1) 
y_true = np.argmax(y_test,axis = 1)

CATEGORIES = ["normal", "PD", "PM"]
CM = confusion_matrix(y_true, pred)

fig, ax = plot_confusion_matrix(conf_mat=CM , figsize=(10,10), class_names=CATEGORIES, cmap=plt.cm.Blues)
plt.show()